In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install transformers datasets evaluate rouge_score

In [3]:
!wget https://github.com/otaviofcoletti/Projeto-Pr-tico-de-PLN/raw/main/data.zip

--2024-06-04 02:59:22--  https://github.com/otaviofcoletti/Projeto-Pr-tico-de-PLN/raw/main/data.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/otaviofcoletti/Projeto-Pr-tico-de-PLN/main/data.zip [following]
--2024-06-04 02:59:22--  https://raw.githubusercontent.com/otaviofcoletti/Projeto-Pr-tico-de-PLN/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16982989 (16M) [application/zip]
Saving to: ‘data.zip.2’

data.zip.2          100%[===================>]  16.20M  87.5MB/s    in 0.2s    

2024-06-04 02:59:22 (87.5 MB/s) - ‘data.zip.2’ saved [16982989/16982989]



In [4]:
!unzip "/content/data.zip"

Archive:  /content/data.zip
replace test/ects/AAN_q3_2021.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
import os
from datasets import Dataset, DatasetDict

def load_text_and_summaries(text_dir, summary_dir):
    data = {"text": [], "summary": [], "title": []}

    # List all files in the text directory
    text_files = os.listdir(text_dir)
    summary_files = os.listdir(summary_dir)

    # Ensure both directories have the same files
    assert set(text_files) == set(summary_files), "Text and summary directories must have matching files"

    for filename in text_files:
        # Read the text file
        with open(os.path.join(text_dir, filename), 'r', encoding='utf-8') as text_file:
            text = text_file.read()

        # Read the summary file
        with open(os.path.join(summary_dir, filename), 'r', encoding='utf-8') as summary_file:
            summary = summary_file.read()

        # Extract the title from the filename (assuming the title is the filename without extension)
        title = os.path.splitext(filename)[0]

        # Append to the data dictionary
        data["text"].append(text)
        data["summary"].append(summary)
        data["title"].append(title)

    return data

def create_dataset_dict(train_dir, test_dir):
    dataset_dict = {}

    for split in ['train', 'test', 'val']:
        text_dir = os.path.join(train_dir if split == 'train' else test_dir, 'ects')
        summary_dir = os.path.join(train_dir if split == 'train' else test_dir, 'gt_summaries')

        data = load_text_and_summaries(text_dir, summary_dir)
        dataset_dict[split] = Dataset.from_dict(data)

    return DatasetDict(dataset_dict)

# Define the directories
train_directory = 'train'
test_directory = 'test'
test_directory = 'val'

# Create the DatasetDict
dataset_dict = create_dataset_dict(train_directory, test_directory)

# Print the dataset_dict to check
print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1681
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 249
    })
    val: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 249
    })
})


In [6]:
dataset_dict['test'][0]

{'text': "I'm joined here today by Steve Westhoven, our President and CEO; Pat Migliaccio, our Chief Financial Officer; as well as other members of our senior management team.\nThis could cause results to materially differ from our expectations, as found in slide one.\nWe will also be referring to certain non-GAAP measures such as net financial earnings or NFE.\nWe believe that NFE provides a more complete understanding of our financial performance.\nHowever, it is not intended to be a substitute for GAAP.\nOur non-GAAP financial measures are discussed more fully in Item seven of our 10-K.\nOur agenda for today is found on slide two.\nSteve will begin today's call with highlights from the quarter, followed by Pat, who will review our financial results.\nNew Jersey Resources delivered strong performance in the first quarter.\nAnd on slide three, I'll take you through the highlights.\nWe reported NFE of $0.46 per share, driven by the performance of our core business, New Jersey Natural G

In [7]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    print(inputs)
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_ects = dataset_dict.map(preprocess_function, batched=True)


Map:   0%|          | 0/1681 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Map:   0%|          | 0/249 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Map:   0%|          | 0/249 [00:00<?, ? examples/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
#(tokenized_ects['train'])

In [11]:
#(tokenized_ects['train'][0]['labels'])

In [12]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [13]:
import evaluate

rouge = evaluate.load("rouge")

In [14]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [15]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [16]:
!pip install accelerate -U

In [17]:
!pip install transformers[torch]

In [18]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [19]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_ects_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)



In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ects["train"],
    eval_dataset=tokenized_ects["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#trainer.train()

In [21]:
#trainer.push_to_hub()


In [22]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="Otavio12/my_awesome_ects_model")

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [23]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Otavio12/my_awesome_ects_model")
model = AutoModelForSeq2SeqLM.from_pretrained("Otavio12/my_awesome_ects_model")

In [46]:
text = "summarize: " + dataset_dict['test'][0]['text'][1000:]

In [47]:
text

"summarize: firming our NFE guidance for fiscal 2021 of $1.55 to $1.65 per share and increasing our fiscal 2022 NFE guidance to $2.20 to $2.30 per share, an increase of $0.15 per share from prior guidance.\nAt New Jersey Natural Gas, we completed almost 90% of the Southern Reliability Link and expect to place the project into service this year.\nWe received approval to move forward with our infrastructure Investment Program, a 5-year, $150 million accelerated recovery program that will improve the resiliency and reliability of our natural gas infrastructure.\nAnd we filed for SAVEGREEN 2020, a new energy efficiency program that is designed to help our customers reduce their energy consumption and save money.\nAt Clean Energy Ventures, we acquired the 2.9 megawatt Mt. Laurel Solar Facility, which is part of our plan to invest $165 million this year.\nIn our Storage & Transportation business, we've begun to construct -- to convert the Southern portion of Adelphia Gateway to natural gas a

In [48]:
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).input_ids


In [49]:
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)


In [50]:
tokenizer.decode(outputs[0], skip_special_tokens=True)


'nj natural gas ffa guidance for fiscal 2021 is $1.55 to $1.65. nj natural gas ffa guidance is expected to increase to $2.20 to $2.30 per share. nj natural gas ffa guidance is expected to increase to $2.20 to $2.30 per share.'

In [51]:
dataset_dict['test'][0]['summary']

'new jersey reaffirms nfe/share outlook of $1.55 to $1.65 for 2021.\nreaffirmed nfe per share (nfeps) guidance of $1.55 to $1.65 for fiscal 2021.\nincreased nfeps guidance for fiscal 2022 by $0.15 to a range of $2.20 to $2.30.\nqtrly consolidated net financial earnings of $44.7 million, or $0.46 per share.\n'